# Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0 and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible.

In [ ]:
import numpy as np

In [ ]:
class Game():
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.maze = (np.random.rand(self.height, self.width) * 10).astype('int')
        self.start = self.maze[0][0]
        self.end = self.maze[self.height - 1][self.width - 1]
        # Pointer
        self.pointer_x = 0
        self.pointer_y = 0
        self.pointer_val = self.maze[0][0]
        self.pointer = f"Currently at {self.start}, located on x={0}, y={0}"
        # Records
        self.steps_taken = 0
        self.time_spent = 0
    
    def move_pointer(self, x_pos, y_pos):
        # Check if x_pos and y_pos are legal (step size of 1 unit, that does not exceed maze boundary)
        if(abs(x_pos - self.pointer_x) > 1 or x_pos >= self.width):
            print("X Step too large")
            return
        if(abs(y_pos - self.pointer_y) > 1 or y_pos >= self.height):
            print("Y Step too large")
            return
        # Check that step is not diagonal (i.e. it is not the case that both x and y positions have changed)
        if((x_pos != self.pointer_x) and (y_pos != self.pointer_y)):
            print("Cannot move diagonally!")
            return
        # If all is legal, make the step
        # Update records
        self.steps_taken += 1
        self.time_spent += self.pointer_val
        # Move pointer
        self.pointer_x = x_pos
        self.pointer_y = y_pos
        self.pointer_val = self.maze[y_pos][x_pos]
        self.pointer = f"Currently at {self.pointer_val}, located on x={x_pos}, y={y_pos}"
        print(f"{self.pointer}")
        
    
    def reset(self):
        print("Reseting game")
        self.pointer_x = 0
        self.pointer_y = 0
        self.pointer = f"Currently at {self.start}, located on x={0}, y={0}"

In [ ]:
game = Game(width=4, height=8)
print(game.maze)

In [ ]:
print(f"Start at {game.start}... going to {game.end}")
print(game.pointer)
game.move_pointer(x_pos=1, y_pos=0)
game.move_pointer(x_pos=2, y_pos=0)
game.move_pointer(x_pos=1, y_pos=0)

In [ ]:
print(game.pointer_val)
print(game.steps_taken)
print(game.time_spent)